# 유튜브 댓글 추출

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import time

In [3]:
#추출할 URL을 리스트형식으로 저장
URLlist = ['https://www.youtube.com/watch?v=FMEOsDqp37M&t=24s',
'https://www.youtube.com/watch?v=riQGjcw4sMk',
'https://www.youtube.com/watch?v=ES1AAH8rBxE&t=4s',
'https://www.youtube.com/watch?v=btvdxtq7Xxc',
'https://www.youtube.com/watch?v=OOMvbwr7HsE',
'https://www.youtube.com/watch?v=a2rj2gSsgbk&t=11s',
'https://www.youtube.com/watch?v=JxG3PXhpDSQ&t=8s',
'https://www.youtube.com/watch?v=NyGBrZcEVEI']

In [ ]:
for i in URLlist:
    #driver초기화
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(3)
    #댓글 추출할 유튜브 페이지 띄움
    driver.get(i)
    #사이즈를 이렇게 하지않으면 오류가 나올 수 있음
    driver.set_window_size(width = 784, height = 831)
    # 첫 화면에 댓글이 없으므로 스크롤 한 번 내려줌
    driver.execute_script("window.scrollTo(0,100000)")
    time.sleep(5)

    switch = True
    beforelocation = 0
    afterlocation = 1
    while switch :
        #내리기 전 과 내린 후의 위치가 같으면 더 이상 댓글이 없는 것

        if beforelocation != afterlocation:
            #스크롤 내리기 전 댓글개수 추출
            comments = len(driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer'))
            #스크롤 내리기 전 마지막 댓글 위치 
            beforelocation = driver.find_element_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer[{}]'.format(comments)).location['y']
            #스크롤 내려줌
            driver.execute_script("window.scrollTo(0,{})".format(beforelocation+10000))
            time.sleep(5)
            #스크롤 내린 후 마지막 댓글 위치
            comments = len(driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer'))
            afterlocation = driver.find_element_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer[{}]'.format(comments)).location['y']
        elif beforelocation == afterlocation:
            switch = False
            break
        else:
            pass

    # 댓글이 모두 나온 페이지의 html 파일 추출
    html = driver.page_source
    time.sleep(5)
    soup = BeautifulSoup(html,'html.parser')
    time.sleep(2)
    commentdata = soup.find_all('yt-formatted-string',{'id':'content-text'})
    
    #댓글들을 하나의 문자열로 저장
    text = ''
    for i in commentdata:
        text = text + i.text + '.'
    #txt로 저장
    #window 파일이름에 들어갈 수 없는 글자들 제거
    name = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
    name = name.replace('?','')
    name = name.replace('|','')
    name = name.replace('*','')
    name = name.replace('>','')
    name = name.replace('<','')
    name = name.replace(':','')
    
    with open('{}.txt'.format(name),'w',encoding = 'UTF-8') as f:
        f.write(text)
    #창 닫아준다.
    driver.close()